In [ ]:
# Celem badania jest wskazanie związku pomiędzy charakterystykami EKG/EDA, a akcjami wykonywanymi w grach.

In [ ]:
# install biosignalsplux package
!pip install biosignalsnotebooks
# install also system libraries necessary for libmagic
!apt-get install libmagic-dev
# install also BioSPPy (we will use them at the end of the lab)
!pip install biosppy
# install biosppy and neurokit2 (will be used at the end of the notebook)
!pip install biosppy neurokit2

In [ ]:
# necessary imports (as before)
import numpy as np
import pandas as pd
from scipy import signal

import biosignalsnotebooks as bsnb

import bokeh
from bokeh.plotting import figure, show
bokeh.io.output_notebook()

import pywt
import copy
import sklearn.mixture
import scipy.stats
import csv
import matplotlib.pyplot as plt
import pandas as pd

In [ ]:
data = pd.read_csv("SUB119-BioSigs.csv")
header = data.columns.tolist()
print(data)
raw_ecg = data['ECG']
raw_eda = data['EDA']
sr = 1000
time = np.linspace(start=0, stop=raw_ecg.shape[0]/sr, num=raw_ecg.shape[0])
print(f"Header:\n{str(header)}\nData:\n{str(data)}")

In [ ]:
p = figure(title='ECG', x_axis_label='Time [s]', y_axis_label='Value [mV]', plot_width=1000,
    plot_height=200)
p.line(time, raw_ecg, legend_label='ECG', line_color='blue')


g = figure(title='EDA', x_axis_label='Time [s]', y_axis_label='Value [µS]', plot_width=1000,
    plot_height=200)
g.line(time, raw_eda, legend_label='EDA', line_color='green')
show(p)
show(g)

In [ ]:

# This is a sumplementary function for Pan-Tompkins plots
def plot_pantompkins(series1, name1, series2, name2, peaks=False):
  colors = bokeh.palettes.brewer['Paired'][3]
  bf = bokeh.plotting.figure(x_axis_label='Time (s)', y_axis_label='ECG (mV)', plot_width=800, plot_height=250)
  bf.line(time, series1, alpha=0.8, line_width=2, color=colors[0], legend_label=name1)
  if peaks:
    bf.circle(time, series2, alpha=0.8, size=10, color='green', legend_label=name2)
  else:
    bf.line(time, series2, alpha=0.8, line_width=2, color='green', legend_label=name2)
  bf.legend.click_policy="hide"
  bf.x_range = bokeh.models.Range1d(10, 20)
  bokeh.plotting.show(bf)

In [ ]:
# Step 1: Band-pass filtering (5-15Hz) to remove noise

from scipy.signal import firwin, lfilter

# Define the filter parameters
low_freq = 5  # Lower cutoff frequency in Hz
high_freq = 15  # Higher cutoff frequency in Hz
filter_order = 4  # Filter order

# Design the band-pass filter using firwin
nyquist_freq = 0.5 * sr  # Nyquist frequency
cutoff_freqs = [low_freq, high_freq]
normalized_cutoffs = [freq / nyquist_freq for freq in cutoff_freqs]
filter_coeffs = firwin(filter_order + 1, normalized_cutoffs, pass_zero=False)

# Apply the band-pass filter to the converted ECG signal
filtered_ecg = lfilter(filter_coeffs, 1.0, raw_ecg)

# We will plot each step to see what we have done
# You can click on the legend to on/off the specific signal
plot_pantompkins(raw_ecg, "ECG signal", filtered_ecg, "Filtered ECG")

In [ ]:
differentiated_ecg = np.ediff1d(filtered_ecg)
plot_pantompkins(filtered_ecg, "Filtered ECG", differentiated_ecg, "Differentiated ECG")

In [ ]:
# Po różniczkowaniu zauważyliśmy, że wartości pików są odwrócone. Oznacza to tyle, że podczas pomiaru elektroda dodadnia została zamieniona z ujemną. W celu jasnej analizy od tego momentu sygnał zostanie odwrócony.

In [ ]:
squared_ecg = flipped_ecg ** 2
plot_pantompkins(flipped_ecg, "Differentiated ECG", squared_ecg, "Squared ECG")

In [ ]:
window_size = int(0.15 * sr)  # Use 0.15 seconds as the window size (adjust as needed)
integration_window = np.ones(window_size)
integrated_ecg = np.convolve(squared_ecg, integration_window, mode='same')
plot_pantompkins(squared_ecg, "Squared ECG", integrated_ecg, "Integrated ECG")

In [ ]:
peaks_found = np.empty(len(raw_ecg))
peaks_found[:] = np.nan
threshold = 30

for x in range(1, len(integrated_ecg)-1):
    # Check if two neighbors have lower values than the central sample in integrated_ecg
    if integrated_ecg[x] > integrated_ecg[x-1] and integrated_ecg[x] > integrated_ecg[x+1]:
        # Check if the corresponding sample in converted_ecg is above the threshold
        if raw_ecg[x] > threshold:
            # We have found a peak!
            peaks_found[x] = raw_ecg[x]
plot_pantompkins(raw_ecg, "ECG signal", peaks_found, "Peaks found", peaks=True)

In [ ]:
# Tachogram
# The determination of peaks is just the beginning. Then, for each adjacent pair of R peaks, we can calculate the distance in time between them. Such a plot is called tachogram.

# It is an interesting plot, on which we can do further analysis (known as HRV analysis).


In [ ]:
peak_times = np.array(time)[~np.isnan(peaks_found)]
print(peak_times)

In [ ]:
tach_values = np.diff(peak_times)
tach_times = peak_times[1:]

print(tach_values)
print(tach_times)

In [ ]:
# Plot the tachogram
bt = bokeh.plotting.figure(x_axis_label='Time (s)', y_axis_label='Cardiac Cycle (s)', plot_width=800, plot_height=250)
bt.line(tach_times, tach_values, alpha=0.8, line_width=2, color='blue')
bokeh.plotting.show(bt)

In [ ]:
tach_values_bsnb, tach_times_bsnb = bsnb.tachogram(raw_ecg, sr, signal=True, out_seconds=True)
bf = bokeh.plotting.figure(x_axis_label='Time (s)',
                           y_axis_label='Cardiac Cycle (s)',
                           plot_width=800,
                           plot_height=250)
bf.line(tach_times, tach_values, legend_label='Your tachogram')
bf.line(tach_times_bsnb, tach_values_bsnb, legend_label='bsnb.tachogram', color='red')
bokeh.plotting.show(bf)

In [ ]:
# plot the data
bf = bokeh.plotting.figure(x_axis_label='Time (s)', y_axis_label='Raw Data', plot_width=800, plot_height=250)
bf.line(time, raw_ecg)
bokeh.plotting.show(bf)
# we do not need to convert values to perform HRV analysis; do you know why?
tach_values, tach_times = bsnb.tachogram(raw_ecg, sr, signal=True, out_seconds=True)
# let's plot the tachogram
bf = bokeh.plotting.figure(x_axis_label='Time (s)', y_axis_label='Cardiac Cycle (s)', plot_width=800, plot_height=250)
bf.line(tach_times, tach_values)
bokeh.plotting.show(bf)

In [ ]:
bpm_values = 60 / np.array(tach_values)
mean_hr = np.mean(bpm_values)
# let's plot the BPM
bf = bokeh.plotting.figure(x_axis_label='Time (s)', y_axis_label='BPM', plot_width=800, plot_height=250)
bf.line(tach_times, bpm_values, legend_label="Heart Rate")
bf.line(tach_times, [mean_hr] * len(tach_times), legend_label="Mean HR")
bokeh.plotting.show(bf)

In [ ]:
# Read the CSV file into a DataFrame
procedure_df = pd.read_csv('SUB119-Procedure.csv', delimiter=';')
print(procedure_df)
# Filter out rows containing "BITalino error" in the EVENT column
filtered_df = procedure_df[~procedure_df['EVENT'].str.contains('BITalino error', na=False)]

# Save the filtered DataFrame to a new CSV file
filtered_df.to_csv('filtered_file.csv', index=False)

In [ ]:


timestamps = []
events = []

with open('filtered_file.csv', 'r') as file:
    csv_reader = csv.reader(file, delimiter=',')
    next(csv_reader)  # Skip header row
    for row in csv_reader:
        timestamp = float(row[0])
        event = row[8]
        if event:
            timestamps.append(timestamp)
            events.append(event)

# Plotting
plt.plot(timestamps, range(len(events)), 'bo')
plt.yticks(range(len(events)), events)
plt.xlabel('Timestamp')
plt.ylabel('Event')
plt.title('Event Timeline')
plt.show()

In [ ]:
df = pd.merge(data, procedure_df, on='TIMESTAMP')
print(df)